In [1]:
import re
import csv
import time
import nltk
import json
import string
import numpy as np
import pandas as pd
from nltk import stem
import sklearn.metrics
from random import randint
from numpy.linalg import norm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#ratings_dataFrame = pd.read_csv("../Preprocess/ratings_dataFrame.csv", index_col=0)

books_dataFrame = pd.read_csv("../Preprocess/books_dataFrame.csv", index_col=0)

#users_dataFrame = pd.read_csv("../Preprocess/users_dataFrame.csv", index_col=0, low_memory=False)


# select only users in userDict rated by more that three users with a non-zero rating

with open('new_userDict.json', 'r') as fp:
    
    new_userDict = json.load(fp)
    
# select only books in isbnDict rated by more that three users with a non-zero rating    
    
with open('new_isbnDict.json', 'r') as fp:
    
    new_isbnDict = json.load(fp)
    
    
with open('dict_row.json', 'r') as fp:
    
    dict_row = json.load(fp)
    
    
with open('dict_col.json', 'r') as fp:
    
    dict_col = json.load(fp)
    

In [2]:
def computeMatrices(train_userDict,train_isbnDict,small_userDict,small_isbnDict, dict_row, dict_col):

    n = len(small_isbnDict)
    m = len(small_userDict)
    
    index = sorted(small_userDict.keys())
    columns = sorted(small_isbnDict.keys())

    dict_row = {k:v for v,k in enumerate(index)}
    dict_col = {k:v for v,k in enumerate(columns)}

    u = np.zeros((m,n)) 
    R = np.zeros((m,n))
    for user in train_userDict:
        for isbn in train_userDict[user]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_userDict[user][isbn]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue

    for isbn in train_isbnDict:
        for user in train_isbnDict[isbn]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_isbnDict[isbn][user]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue
                
    small_utility_DataFrame = pd.DataFrame(u, index = index, columns = columns)
    R = pd.DataFrame(R, index = index, columns = columns)

    return u, R, small_utility_DataFrame

In [3]:
def convert(user_number,book_number, index_user_cluster, index_book_cluster):
    
    try:
        user_cluster = index_user_cluster[user_number]
    except:
        print("key problem")
        return None
    
    if book_number != None:
        
        try:
            book_cluster = index_book_cluster[book_number]
            return user_cluster, book_cluster
        except:
            print("key problem")
            return None
        
    return user_cluster

# clusters2_new_userDict

# key -----> number user cluster
# value ----> books that are been rated by users inside number user cluster

# clusters2_new_isbnDict

# key -----> number book cluster
# value ----> users that have rated books inside number book cluster

In [4]:
with open('clustering/file/clusters2_new_userDict.json', 'r') as fp:
    
    clusters_new_userDict = json.load(fp)
    
    
with open('clustering/file/clusters2_new_isbnDict.json', 'r') as fp:
    
    clusters_new_isbnDict = json.load(fp)
    

In [5]:
with open('clustering/file/clusters_dict_row.json', 'r') as fp:
    
    clusters_dict_row = json.load(fp)
       
with open('clustering/file/clusters_dict_col.json', 'r') as fp:
    
    clusters_dict_col = json.load(fp)
    
print("Ok") 

with open('clustering/index_book_user_clusters/index_user_cluster.json', 'r') as fp:
    
    index_user_cluster = json.load(fp)
    
    
with open('clustering/index_book_user_clusters/index_book_cluster.json', 'r') as fp:
    
    index_book_cluster = json.load(fp)
    
print("Ok")

Ok
Ok


# for the moment we have only number_cluster(item or user) and all the element inside that cluster (users or items); for example inside clusters2_new_isbnDict["1] we have all the users that have rated books belonging at this cluster(the information of what books/users are inside what cluster is inside cluster_book_dict.json and cluster_users_dict.json)

# but to compute the utility matrix I need to have a structure that contains number_cluster(item or user) and as values clusters(users or items) that have rated element inside number_cluster

# cluster_books

# key -----> number book cluster
# value -----> clusters of users that have rated number book cluster(the elements inside this cluster) 

In [6]:
def generateClustersUsingClustersNewDicts(clusters_new_isbnDict, index_user_cluster):

    clusters_books = {k:{} for k in clusters_new_isbnDict}

    for c_i in clusters_new_isbnDict:

        for user in clusters_new_isbnDict[c_i]:

            try:
                c_u = index_user_cluster[user]

                if c_u not in clusters_books[c_i]:

                    clusters_books[c_i][c_u] = []

                if float(clusters_new_isbnDict[c_i][user]) != 0.0:

                    clusters_books[c_i][c_u].append(float(clusters_new_isbnDict[c_i][user]))

            except:
                continue
                
    for c_b in clusters_books:

        users_cluster = list(clusters_books[c_b].keys())

        for c_u in users_cluster:

            if clusters_books[c_b][c_u] == []:

                del clusters_books[c_b][c_u]
            else:
                clusters_books[c_b][c_u] = np.mean(clusters_books[c_b][c_u])
                
                
    return clusters_books

In [7]:
with open('clustering/CLUSTERS_ITEMS.json', 'r') as fp:
    
    CLUSTERS_ITEMS = json.load(fp)
    
    
with open('clustering/CLUSTERS_USERS.json', 'r') as fp:
    
    CLUSTERS_USERS = json.load(fp)

In [ ]:
#clusters_books = generateClustersUsingClustersNewDicts(clusters_new_isbnDict, index_user_cluster)

#clusters_users = generateClustersUsingClustersNewDicts(clusters_new_userDict, index_book_cluster)

In [ ]:
#u_clusters, R_clusters, utility_DataFrame_clusters = computeMatrices(CLUSTERS_USERS,CLUSTERS_ITEMS,
 #                                                                          CLUSTERS_USERS,CLUSTERS_ITEMS,
 #                                                                           clusters_dict_row,clusters_dict_col)

#u, R, utility_DataFrame = computeMatrices(new_userDict,new_isbnDict,new_userDict,new_isbnDict,dict_row,dict_col)

# I want to recommend some books to a certain user  using user/users

# start considering all the books well rated by my choosen user

# item/items collaborative filtering  -----> a book to a certain user

# the idea is that if number_user has given good ratings to books similar to number_book, probably number_user appreciates number_book.

# take all the books rated (positive or negative) by number_user

In [8]:
def booksRatedUser(new_isbnDict, new_userDict, user_number, score):
    
    '''
    input:  new_userDict (Dict), user_number (int), score (int)
    
    action: select all books well rated by my user
    
    output: books_rated (list)
    
    '''
    
    books_rated = []
    
    for book in new_userDict[str(user_number)]:
        
        if int(new_userDict[str(user_number)][book]) > score:
            
            try:
                
                new_isbnDict[book]
                books_rated.append(book)
                
            except:
                
                continue
            
    return list(set(books_rated))


def SimilarityBooks(utility_DataFrame, book_number, books_similar, measure = "euclid"):
    
    '''
    
    input: utility_DataFrame (DataFrame), book_number (int), books_similar (List)
    
    action: compute cosine similarity between book_number and all the books in books_similar
    
    output: new_similarity (List of tuples)
    
    '''

    x = utility_DataFrame[str(book_number)]
    x_length = norm(x)
    
    y = utility_DataFrame[books_similar]
    y_length = norm(utility_DataFrame[books_similar],axis=0)

    
    num = (y.T.values*x.values).sum(axis=1)
    
    if measure == "cos":
        den = x_length*y_length
    else:
        den = 1

    similarity = num/den
    similarity = np.nan_to_num(similarity)
    
    d = list(zip(list(books_similar),similarity))
    new_similarity = sorted(d, key=lambda tup: tup[1], reverse=True)
    
    return new_similarity

def itemItemsRecommendation(new_similarity, user_number, book_number, k, new_userDict):
    
    
    '''
    
    input:  new_similarity(List of tuples), new_isbnDict(Dict), k(int)
    
    action: recommend item using the ratings of similar items
    
    output: recommendation (float)
    
    '''
    
    
    if len(new_similarity) < k:
        
        
        recommendation = np.mean([u[1] for u in new_similarity])
    
    else:
        
        for u in new_similarity[:k]:
            
            recommendation = np.mean([u[1] for u in new_similarity[:k]])
        
    
    return recommendation

def itemItemsScore(new_userDict, new_similarity, k, user_number):
    
    score = [int(new_userDict[str(user_number)][u[0]]) for u in new_similarity[:k]]
    
    if score == []:
        
        print("Not possible to perform recommendatios")
        return None
    
    return np.mean(score)

def CollaborativeFilteringItemItems(utility_DataFrame, new_userDict, new_isbnDict, user_number, book_number, score_min, k):
    
    
    '''
    
    generate the book_number rating for the user_number using similarity between number_book and book rated by number_user   
    
    '''
    
    score = None
    books_rated_user = booksRatedUser(new_isbnDict, new_userDict, user_number,score_min)
    
    
    if books_rated_user == []:
        
        return score
    
    new_similarity = SimilarityBooks(utility_DataFrame, book_number, books_rated_user)
    
    
    if new_similarity == []:

        return score
    
    recommendation = itemItemsRecommendation(new_similarity[1:], user_number, book_number, k, new_userDict)
    
    
    if recommendation == 0.0:
        
        return score
        
        
    score = itemItemsScore(new_userDict, new_similarity[1:], k, user_number)
    
    return score

In [9]:
def mainItemItems_clustering(books_dataFrame, utility_DataFrame_cluster,R_cluster, new_userDict, new_isbnDict, d_user, d_isbn,
                  index_user_cluster, index_book_cluster,
                  score_min = 0, k = 3):
    
    
    print("Please choose a user and/or an item\n")
    print("user or random user: \n")
    user_number = input()
    if user_number == "":
        user_number = None
    print("item or random item: \n")
    book_numbers = input()
    if book_numbers == "":
        book_numbers = None
    
    if user_number != None or book_numbers != None:
        
        if user_number == None:
            
            n = randint(0,len(new_userDict))
            user_number = d_user[n]
            
        if book_numbers == None:
            
            m = randint(0,len(new_isbnDict))
            book_numbers = d_isbn[m]
        
        for book_number in book_numbers.split():
            score = None
            
            score = CollaborativeFilteringItemUsers(utility_DataFrame, new_userDict, new_isbnDict, user_number, book_number,
                                                    score_min, k)
            if score == None:

                user_cluster, book_cluster = convert(user_number,book_number, index_user_cluster, index_book_cluster)

                if R_cluster[book_cluster][user_cluster] == 1:

                    score = utility_DataFrame_cluster[book_cluster][user_cluster]
                    
            if score == None:
                    
                    term1 = utility_DataFrame_cluster[book_cluster]
                    r1 = R_cluster[book_cluster]
                    term1 = term1[r1 ==1]
                    
                    term2 = utility_DataFrame_cluster.loc[user_cluster]
                    r2 = R_cluster.loc[user_cluster]
                    term2 = term2[r2==1]
                    
                    score = (np.mean(term1) + np.mean(term2))/2
                    
            if score == None:
                print("Nothing to recommend")
                
            else:    
                try:
                    print("\nThe predicted rating (using item-based with clusterization) for book <", 
                          books_dataFrame.loc[book_number]["Book-Title"], " > given by user <", user_number, "> is", np.round(score,3),"\n")

                except:
                    print(book_number, user_number, score)
                    
            
        
    else:

        score = None

        while score == None:



            n = randint(0,len(new_userDict))
            m = randint(0,len(new_isbnDict))

            user_number = d_user[n]
            book_number = d_isbn[m]
            
            score = CollaborativeFilteringItemItems(utility_DataFrame, new_userDict, new_isbnDict, user_number, book_number, 
                                                    score_min, k)
            
            if score == None:
            
                user_cluster, book_cluster = convert(user_number,book_number, index_user_cluster, index_book_cluster)
                
                if R_cluster[book_cluster][user_cluster] == 1:
                    
                    score = utility_DataFrame_cluster[book_cluster][user_cluster]
                    
            if score == None:

                    term1 = utility_DataFrame_cluster[book_cluster]
                    r1 = R_cluster[book_cluster]
                    term1 = term1[r1 ==1]

                    term2 = utility_DataFrame_cluster.loc[user_cluster]
                    r2 = R_cluster.loc[user_cluster]
                    term2 = term2[r2==1]

                    score = (np.mean(term1) + np.mean(term2))/2
                    

        try:
            print("The predicted rating (using item-based with clusterization) for book <", 
                  books_dataFrame.loc[book_number]["Book-Title"], " > given by user <", user_number, "> is", np.round(score,3))

        except:
            print(book_number, user_number, score)
        
    #return book_number, user_number, score


# item/users collaborative filtering  -----> a book to a certain user

# the idea is that if number_user is (really) similar to a user that has given a good rating to number_book, probably also number_user appreciates number_book

In [10]:
# ATTENTION!!!  different respect to usersHaveRatedBook(s)

def usersHaveRatedBook(new_isbnDict, new_userDict, book_number, score):
    
    
    '''
    
    input:  new_isbnDict (Dict), new_userDict (Dict), book_number (int), score (int)
    
    action: select all users that have given a good rating to book_number
    
    output: users_rated (list)
    
    '''
    
    users_rated = []
    
    for user in new_isbnDict[str(book_number)]:
        
        if int(new_isbnDict[str(book_number)][user]) > score:
            
            try:
                
                new_userDict[user]
                users_rated.append(user)
                
            except:
                
                continue
            
    return list(set(users_rated))


#users_rated_book = usersHaveRatedBook(new_isbnDict, new_userDict, book_number, 5)

def SimilarityUsers(utility_DataFrame, user_number, users_similar, measure = "euclid"):
    
    '''
    
    input: utility_DataFrame (DataFrame), user_number (int), user_similar (List)
    
    action: compute cosine similarity between user_number and all the user in users_similar
    
    output: new_similarity (List of tuples)
    
    '''

    x = utility_DataFrame.loc[str(user_number)]
    x_length = norm(x)
    
    y = utility_DataFrame.loc[users_similar]
    y_length = norm(utility_DataFrame.loc[users_similar],axis=1)

    
    num = (y.values*x.values).sum(axis=1)
    
    if measure == "cos":
        den = x_length*y_length
    else:
        den = 1

    similarity = num/den
    similarity = np.nan_to_num(similarity)
    
    d = list(zip(list(users_similar),similarity))
    new_similarity = sorted(d, key=lambda tup: tup[1], reverse=True)
    
    
    return new_similarity

def ItemUsersRecommendation(new_similarity, new_userDict, k):
    
    '''
    
    input:  new_similarity(List of tuples), new_userDict(Dict), score(int), k(int)
    
    action: recommend items using the ratings of similar users
    
    output: recommendation (Dict), books (Dict)
    
    '''
    
    if new_similarity == []:
        
        return None
    
    if len(new_similarity) > k:
    
        recommendation = np.mean([u[1] for u in new_similarity[:k]]) 
    
    else:
        
        recommendation = np.mean([u[1] for u in new_similarity]) 
        
    return recommendation

def itemUsersScore(new_similarity,new_isbnDict, book_number, k):
    
    if new_similarity == []:
        
        return None
    
    if len(new_similarity) > k:
        
        score = [int(new_isbnDict[str(book_number)][u[0]]) for u in new_similarity[:k] if u[1] !=0.0]
        
    else:
        
        score = [int(new_isbnDict[str(book_number)][u[0]]) for u in new_similarity if u[1] !=0.0]
    
    return np.mean(score)

def CollaborativeFilteringItemUsers(utility_DataFrame, new_userDict, new_isbnDict, user_number, book_number, score_min = 0, k = 3):
    
    score = None
    users_rated_book = usersHaveRatedBook(new_isbnDict, new_userDict, book_number, score_min)
    
    
    if users_rated_book == []:
        
        return score
    
    
    new_similarity = SimilarityUsers(utility_DataFrame, user_number, users_rated_book)
    
    
    if new_similarity == []:

        return score
    
    recommendation = ItemUsersRecommendation(new_similarity, new_userDict, k)
    
        
    if recommendation == 0.0:
        
        return score
        
    score = itemUsersScore(new_similarity,new_isbnDict, book_number, k)
    
        
    return score

In [11]:
def mainItemUsers_clustering(books_dataFrame, utility_DataFrame_cluster, R_cluster, new_userDict, new_isbnDict, d_user, d_isbn,
                  index_user_cluster, index_book_cluster,
                  score_min = 0, k = 3):
    
    
    print("Please choose a user and/or an item\n")
    print("user or random user: \n")
    user_number = input()
    if user_number == "":
        user_number = None
    print("item or random item: \n")
    book_numbers = input()
    if book_numbers == "":
        book_numbers = None
    
    if user_number != None or book_numbers != None:
        
        if user_number == None:
            
            n = randint(0,len(new_userDict))
            user_number = d_user[n]
            
        if book_numbers == None:
            
            m = randint(0,len(new_isbnDict))
            book_numbers = d_isbn[m]
        
        for book_number in book_numbers.split():
            
            score = None
            
            score = CollaborativeFilteringItemUsers(utility_DataFrame, new_userDict, new_isbnDict, user_number, book_number,
                                                    score_min, k)
            if score == None:

                user_cluster, book_cluster = convert(user_number,book_number, index_user_cluster, index_book_cluster)

                if R_cluster[book_cluster][user_cluster] == 1:

                        score = utility_DataFrame_cluster[book_cluster][user_cluster]
                        
                if score == None:

                    term1 = utility_DataFrame_cluster[book_cluster]
                    r1 = R_cluster[book_cluster]
                    term1 = term1[r1 ==1]

                    term2 = utility_DataFrame_cluster.loc[user_cluster]
                    r2 = R_cluster.loc[user_cluster]
                    term2 = term2[r2==1]

                    score = (np.mean(term1) + np.mean(term2))/2
                    
            if score == None:
                print("Nothing to recommend")
            else:
                try:
                    print("\nThe predicted rating (using user-based with clusterization) for book <", 
                      books_dataFrame.loc[str(book_number)]["Book-Title"], "> given by user <", user_number, "> is", np.round(score,3), "\n")

                except:
                    print(book_number, user_number, score)
           
                
    else:
            
        score = None

        while score == None:


            n = randint(0,len(new_userDict))
            m = randint(0,len(new_isbnDict))

            user_number = d_user[n]
            book_number = d_isbn[m]
            

            score = CollaborativeFilteringItemUsers(utility_DataFrame, new_userDict, new_isbnDict, user_number, book_number,
                                                    score_min,k)
            if score == None:
            
                user_cluster, book_cluster = convert(user_number,book_number, index_user_cluster, index_book_cluster)
                
                if R_cluster[book_cluster][user_cluster] == 1:
                    
                    score = utility_DataFrame_cluster[book_cluster][user_cluster]
                    
                if score == None:

                    term1 = utility_DataFrame_cluster[book_cluster]
                    r1 = R_cluster[book_cluster]
                    term1 = term1[r1 ==1]

                    term2 = utility_DataFrame_cluster.loc[user_cluster]
                    r2 = R_cluster.loc[user_cluster]
                    term2 = term2[r2==1]

                    score = (np.mean(term1) + np.mean(term2))/2
                    
        try:

            print("The predicted rating (using user-based with clusterization) for book <", 
              books_dataFrame.loc[str(book_number)]["Book-Title"], "> given by user <", user_number, "> is", np.round(score,3))

        except:

            print(book_number, user_number, score)

    
    
    #return book_number, user_number, score
    

In [12]:
def mainClustering(books_dataFrame, utility_DataFrame_cluster,R_cluster, new_userDict, new_isbnDict, d_user, d_isbn,
                  index_user_cluster, index_book_cluster):
    
    
    print("Please choose a user and/or an item\n")
    print("user or random user: \n")
    user_number = input()
    if user_number == "":
        user_number = None
    print("item or random item: \n")
    book_numbers = input()
    if book_numbers == "":
        book_numbers = None
    
    if user_number != None or book_numbers != None:
        
        if user_number == None:
            
            n = randint(0,len(new_userDict))
            user_number = d_user[n]
            
        if book_numbers == None:
            
            m = randint(0,len(new_isbnDict))
            book_numbers = d_isbn[m]
        
        for book_number in book_numbers.split():
            
            try:
                score = None
                user_cluster, book_cluster = convert(user_number,book_number, index_user_cluster, index_book_cluster)
                if R_cluster[book_cluster][user_cluster] == 1:
                        score = utility_DataFrame_cluster[book_cluster][user_cluster]
                        
                if score == None:
                    
                    term1 = utility_DataFrame_cluster[book_cluster]
                    r1 = R_cluster[book_cluster]
                    term1 = term1[r1 ==1]
                    
                    term2 = utility_DataFrame_cluster.loc[user_cluster]
                    r2 = R_cluster.loc[user_cluster]
                    term2 = term2[r2==1]
                    
                    score = (np.mean(term1) + np.mean(term2))/2
                        
                if score == None:
                    print("Nothing to recommend")
                else:
                    try:
                        print("\nThe predicted rating (using user-based with clusterization) for book <", 
                          books_dataFrame.loc[str(book_number)]["Book-Title"], "> given by user <", user_number, "> is", 
                              np.round(score,3),"\n")
                    except:
                        print(book_number, user_number, score)
                
            except:
                continue
                    
    else:
            
        score = None

        while score == None:


            n = randint(0,len(new_userDict))
            m = randint(0,len(new_isbnDict))

            user_number = d_user[n]
            book_number = d_isbn[m]
            
            try:
                user_cluster, book_cluster = convert(user_number,book_number, index_user_cluster, index_book_cluster)

                if R_cluster[book_cluster][user_cluster] == 1:

                    score = utility_DataFrame_cluster[book_cluster][user_cluster]
                    
                if score == None:

                    term1 = utility_DataFrame_cluster[book_cluster]
                    r1 = R_cluster[book_cluster]
                    term1 = term1[r1 ==1]

                    term2 = utility_DataFrame_cluster.loc[user_cluster]
                    r2 = R_cluster.loc[user_cluster]
                    term2 = term2[r2==1]

                    score = (np.mean(term1) + np.mean(term2))/2
                    
            except:
                continue
        try:

            print("The predicted rating (using user-based with clusterization) for book <", 
              books_dataFrame.loc[str(book_number)]["Book-Title"], "> given by user <", user_number, "> is", np.round(score,3))

        except:

            print(book_number, user_number, score)

    
    
    #return book_number, user_number, score
    

In [13]:
def mainCollaborativeFiltering_CLUSTERING(books_dataFrame, utility_DataFrame_cluster,R_cluster, new_userDict, new_isbnDict,
                                          index_user_cluster, index_book_cluster,
                                          d_user = None, d_isbn = None, score_min = 0, k = 3):
    
    n = -1
    
    d_user = sorted(new_userDict.keys())
    d_isbn = sorted(new_isbnDict.keys())
    
    while n not in [1,2,3]:
    
        print("What method do you want to try?\n")

        print("1 CollaborativeFilteringItemItems using clustering")
        print("2 CollaborativeFilteringItemUsers using clustering")
        print("3 CollaborativeFiltering clustering approach")
        
        try:
            n = int(input())
        except:
            continue
        
    if n == 1:
    
        mainItemItems_clustering(books_dataFrame, utility_DataFrame_cluster,R_cluster,new_userDict, new_isbnDict, d_user, d_isbn,
                      index_user_cluster, index_book_cluster,
                      score_min, k)
    elif n == 2:
    
        mainItemUsers_clustering(books_dataFrame, utility_DataFrame_cluster,R_cluster,new_userDict, new_isbnDict, d_user, d_isbn,
                      index_user_cluster, index_book_cluster,
                      score_min, k)
        
    elif n ==3:
        
        mainClustering(books_dataFrame, utility_DataFrame_cluster,R_cluster, new_userDict, new_isbnDict, d_user, d_isbn,
                  index_user_cluster, index_book_cluster)

In [14]:
u_cluster, R_cluster, utility_DataFrame_cluster = computeMatrices(CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                           CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col)


u, R, utility_DataFrame = computeMatrices(new_userDict,new_isbnDict,new_userDict,new_isbnDict,dict_row,dict_col)

In [15]:
r = -1

mainCollaborativeFiltering_CLUSTERING(books_dataFrame, utility_DataFrame_cluster,R_cluster, new_userDict, new_isbnDict,
                                          index_user_cluster, index_book_cluster)

while True:
    
    print("\nAnother try?\n")
    print("y")
    print("n")
    
    r = input()
    
    if r == "y":
        mainCollaborativeFiltering_CLUSTERING(books_dataFrame, utility_DataFrame_cluster,R_cluster, new_userDict, new_isbnDict,
                                          index_user_cluster, index_book_cluster)
        
    elif r == "n":
        
        print("Bye")
        break

What method do you want to try?

1 CollaborativeFilteringItemItems using clustering
2 CollaborativeFilteringItemUsers using clustering
3 CollaborativeFiltering clustering approach
1
Please choose a user and/or an item

user or random user: 


item or random item: 


The predicted rating (using item-based with clusterization) for book < Per Anhalter durch die Galaxis.  > given by user < 231612 > is 8.0

Another try?

y
n
y
What method do you want to try?

1 CollaborativeFilteringItemItems using clustering
2 CollaborativeFilteringItemUsers using clustering
3 CollaborativeFiltering clustering approach

What method do you want to try?

1 CollaborativeFilteringItemItems using clustering
2 CollaborativeFilteringItemUsers using clustering
3 CollaborativeFiltering clustering approach
3
Please choose a user and/or an item

user or random user: 


item or random item: 


The predicted rating (using user-based with clusterization) for book < The Lion, the Witch and the Wardrobe (rpkg) (Narnia) > 

In [ ]:
new_userDict["120908"]